In [ ]:
from bach_dataset import get_bach_test_dataset
from dataset import load_dataloader
from model import init_model, load_model
import numpy as np
import matplotlib.pyplot as plt
import torch
from tqdm import tqdm
import pandas as pd

%load_ext autoreload
%autoreload 2


In [ ]:
dataset = get_bach_test_dataset()
DEVICE = "cuda"
classes = ["Benign", "InSitu", "Invasive", "Normal"]
class_map = {"Benign": 2, "InSitu": 3, "Invasive": 4, "Normal": 1}
weights_path = "weights_BACH_LONG_2.pt"
dataloader = load_dataloader(dataset, 1, classes=len(classes), shuffle=False)


In [ ]:
model = init_model(len(classes), None, teacher_checkpoint=True, mode="normal")
for parameter in model.parameters():
    parameter.requires_grad = False
weights = torch.load(weights_path)
model.load_state_dict(weights)
model.eval()
model = model.to(DEVICE)

In [ ]:
preds = np.zeros(len(dataset), dtype=int)
for index, batch in (pbar := tqdm(enumerate(dataloader), total=len(dataloader), dynamic_ncols=True, colour="red")):
    batch = batch.to(DEVICE)
    class_index = torch.argmax(model(batch).cpu()[0])
    alf_class = classes[class_index]
    preds[index] = class_map[alf_class]

In [ ]:
df = pd.DataFrame({"case": np.arange(0, 100), "class": preds})
df

In [ ]:
df.to_csv("submissions/fifth.csv", index=False)

In [ ]:
def compare_submissions(df1: pd.DataFrame, df2: pd.DataFrame):
    preds1 = df1["class"].to_numpy()
    preds2 = df2["class"].to_numpy()

    print(np.unique(preds1, return_counts=True))
    print(np.unique(preds2, return_counts=True))
    total = (preds1==preds2).astype(int).mean()
    print(np.argwhere(preds1!=preds2))
    return total

In [ ]:
first = pd.read_csv("submissions/third.csv")
second = pd.read_csv("submissions/fifth.csv")

In [ ]:
score = compare_submissions(first, second)
print(first.iloc[29])
print(second.iloc[29])
print("Similiarity", score)